In [1]:
import time
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [62]:
batch = 32
lr = 1e-3
num_epochs = 25

In [3]:
df = pd.read_csv("../nesg/nesg.csv")

fl = open("../nesg/nesg.fasta")
dic1 = {}
chro = fl.readline().strip().split('>')[1]
seq1 = ''
for i in fl:
    if '>' in i:
        dic1[chro] = seq1
        chro = i.strip().split('>')[1]
        seq1 = ''
    else:
        seq1 += i.strip()
dic1[chro] = seq1
fl.close()

for j in range(len(df)):
    if df.iloc[j, 2] > 0:
        df.iloc[j, 2] = 1

In [4]:
seq = list(dic1.values())
df['seq'] = seq
data = df[['seq', 'sol']].values    # ndarray

In [5]:
def get_set(ddata):
    """return three list: test_set, train_set, valid_set"""
    f_set = []
    test_set = []   # 测试集：1941
    train_set = []  # 训练集：5821
    valid_set = []  # 验证集：1941
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for f_index, test_index in split.split(ddata[:, :-1], ddata[:, -1]):
        f_set = ddata[f_index, :]
        test_set = ddata[test_index, :]

    split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
    for train_index, valid_index in split.split(f_set[:, :-1], f_set[:, -1]):
        train_set = f_set[train_index, :]
        valid_set = f_set[valid_index, :]

    return test_set, train_set, valid_set


def get_sol(d):
    """sol: return a tensor variable"""
    sol = d[:, 1]
    sol = sol.reshape(-1, 1)
    sol_onehotencoder = OneHotEncoder(sparse=False)
    sol_onehotencoded = sol_onehotencoder.fit_transform(sol)
    sol_onehotencoded = torch.from_numpy(sol_onehotencoded)
    return (sol_onehotencoded).float()


def get_seq(d):
    """given an array d, return a tensor with (d.shape[0],360,20)"""
    dic = {'X': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'A': (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'C': (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'D': (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'E': (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'F': (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'G': (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'H': (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'I': (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'K': (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'L': (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'M': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0),
           'N': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0),
           'P': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
           'Q': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0),
           'R': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0),
           'S': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
           'T': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0),
           'V': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
           'W': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
           'Y': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)}

    ds = np.zeros((1, 360, 20))
    for i in d[:, 0]:
        dds = np.zeros((1, 20))
        for j in i:
            dds = np.insert(dds, len(dds), dic[j], axis=0)
            dds = dds[1:dds.shape[0]]

            if dds.shape[0] <= 360:
                dds = np.pad(dds, ((0, 360 - dds.shape[0]), (0, 0)), 'constant', constant_values=(0, 0))
            else:
                dds = dds[0:360]

        ds = np.insert(ds, len(ds), dds, axis=0)

    ds = ds[1:ds.shape[0]]

    return (torch.tensor(ds)).float()


def get_data(d):
    x = get_seq(d)
    y = get_sol(d)
    ds = TensorDataset(x, y)
    dl = DataLoader(ds, batch_size=batch, shuffle=True)
    return dl

In [6]:
test, train, valid = get_set(data)
(pd.DataFrame(test, columns=['seq', 'sol'])).to_csv("../data/test_set.csv", index=0)
(pd.DataFrame(train, columns=['seq', 'sol'])).to_csv("../data/train_set.csv", index=0)
(pd.DataFrame(valid, columns=['seq', 'sol'])).to_csv("../data/valid_set.csv", index=0)

In [7]:
test_set = get_data(test)
train_set = get_data(train)
valid_set = get_data(valid)

In [8]:
class YModel(nn.Module):
    def __init__(self):
        super(YModel, self).__init__()

        self.lstm = nn.LSTM(input_size=20, hidden_size=128, num_layers=2, bidirectional=True, dropout=0.5, batch_first=True)
        self.fc = nn.Linear(128*2, 2)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

In [53]:
def train_model(model):
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')

    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    val_acc_history = []
    train_acc_history = []

    since = time.time()
    for epoch in range(num_epochs):
        train_losses = []
        valid_losses = []
        train_acc = []
        valid_acc = []
#         running_loss = 0.0
#         running_corrects = 0
        model.train()
        for iteration, (train_x, train_y) in enumerate(train_set):
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            train_out = model(train_x)
            train_loss = criterion(train_out, train_y)
#             running_loss += train_loss.item()
            train_losses.append(train_loss.item())

            pred = train_out.argmax(dim=1)
            correct = torch.eq(pred, train_y.argmax(dim=1)).float()
            train_acc.append((correct.sum()/len(correct)).item())
#             running_corrects += torch.eq(pred, train_y.argmax(dim=1)).sum().item()
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
        
        print("train loss:{}".format(np.array(train_losses).mean()))
        print("train acc:{}".format(np.array(train_acc).mean()))
#         print("train loss:{}".format(running_loss / len(train_set)))
#         print("train acc:{}".format(running_corrects / len(train_set) / batch))

        

#         running_loss = 0.0
#         running_corrects = 0
        model.eval()
        for iteration, (valid_x, valid_y) in enumerate(valid_set):
            valid_x = valid_x.to(device)
            valid_y = valid_y.to(device)
            valid_out = model(valid_x)
            valid_loss = criterion(valid_out, valid_y)
#             running_loss += valid_loss.item()
            valid_losses.append(valid_loss.item())
            pred = valid_out.argmax(dim=1)
            correct = torch.eq(pred, valid_y.argmax(dim=1)).float()
            valid_acc.append((correct.sum()/len(correct)).item())
#             running_corrects += torch.eq(pred, valid_y.argmax(dim=1)).sum().item()
#         print("valid loss:{}".format(running_loss / len(valid_set)))
#         print("valid acc:{}".format(running_corrects / len(valid_set) / batch))
        print("valid loss:{}".format(np.array(valid_losses).mean()))
        print("valid acc:{}".format(np.array(valid_acc).mean()))
        print("epoch:{} {}s\n".format(epoch, (since-time.time()) % 60))
        since = time.time()

#     running_loss = 0.0
#     running_corrects = 0
    test_losses = []
    test_acc = []
    for iteration, (test_x, test_y) in enumerate(test_set):
        test_x = test_x.to(device)
        test_y = test_y.to(device)
        test_out = model(test_x)
        test_loss = criterion(test_out, test_y)
#         running_loss += test_loss.item()
        test_losses.append(test_loss.item())
        pred = test_out.argmax(dim=1)
        correct = torch.eq(pred, test_y.argmax(dim=1)).float()
        test_acc.append((correct.sum()/len(correct)).item())
    print("test loss:{}".format(np.array(test_losses).mean()))
    print("test acc:{}".format(np.array(test_acc).mean()))
#         running_corrects += torch.eq(pred, test_y.argmax(dim=1)).sum().item()
#     print("test loss:{}".format(running_loss / len(test_set)))
#     print("test acc:{}".format(running_corrects / len(test_set) / batch))

    torch.save(obj=model.state_dict(), f="models/YModel.pth")

In [63]:
model = YModel()
train_model(model)

train loss:0.6719748019189625
train acc:0.6059541420622186
valid loss:0.670609893368893
valid acc:0.6078893444577201
epoch:0 36.99095892906189s

train loss:0.6704661033965729
train acc:0.6076315512041469
valid loss:0.6699977282617913
valid acc:0.6045081967213115
epoch:1 36.94128227233887s

train loss:0.6684827393555379
train acc:0.6065881235913916
valid loss:0.6682557820296678
valid acc:0.6067622951308235
epoch:2 36.991349935531616s

train loss:0.6677142220375302
train acc:0.6054786559972134
valid loss:0.6703254297131398
valid acc:0.6056352459260674
epoch:3 37.14221668243408s

train loss:0.6660457508904594
train acc:0.6063900043021192
valid loss:0.6689599859909933
valid acc:0.6067622951308235
epoch:4 37.09675335884094s

train loss:0.664578514112221
train acc:0.6074598479074437
valid loss:0.6692869423842821
valid acc:0.6090163935403354
epoch:5 36.959115982055664s

train loss:0.664349899842189
train acc:0.607420224082339
valid loss:0.6715338010279859
valid acc:0.605430327966565
epoch:6 3

In [32]:
mytest0 = torch.tensor([
              [
                  [1., 5., 5., 2.],
                  [9., -6., 2., 8.],
                  [-3., 7., -9., 1.]
              ],
 
              [
                  [-1., 7., -5., 2.],
                  [9., 6., 2., 8.],
                  [3., 7., 9., 1.]
              ]], requires_grad=True)
mytest1 = mytest0.argmax(dim=1)
    

In [34]:
print(mytest0.requires_grad)
print(mytest1.requires_grad)

True
False


In [48]:
my = []
for iteration, (test_x, test_y) in enumerate(test_set):
    t = test_y.argmax(dim=1)
    print(t)
    print(t.requires_grad)
    print((torch.eq(t, t).float()).dtype)
    x = torch.eq(t, t).float()
    print((x.sum()/len(x)).item())
    my.append(x.sum()/len(x))
    break

print(np.array(my).mean())    
print(train_set)    
print(len(test_set))

tensor([1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0])
False
torch.float32
1.0
1.0
122
